<a href="https://colab.research.google.com/github/Saoudyahya/Image-Debt-Estimation-Model/blob/main/Dept_estimation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets h5py scipy -q

In [ ]:
# Simple version - install packages first
!pip install datasets transformers torch torchvision -q

# Quick test to make sure datasets work
from datasets import load_dataset
dataset = load_dataset("sayakpaul/nyu_depth_v2")
print(f"Dataset loaded: {len(dataset['train'])} samples")

# Then run the main training code

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os
import h5py
import requests
import zipfile
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Alternative: Use HuggingFace datasets for reliable download
def load_nyu_depth_v2_hf():
    """Load NYU Depth V2 using HuggingFace datasets - more reliable"""
    try:
        from datasets import load_dataset
        print("Loading NYU Depth V2 from HuggingFace...")
        dataset = load_dataset("sayakpaul/nyu_depth_v2")
        return dataset
    except ImportError:
        print("HuggingFace datasets not installed. Install with: pip install datasets")
        return None

# Manual download with better error handling
def download_nyu_depth_v2(data_dir='./nyu_depth_v2'):
    """Download and extract NYU Depth V2 dataset with better error handling"""
    os.makedirs(data_dir, exist_ok=True)

    # Multiple mirror URLs (original is often unreliable)
    urls = [
        'http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labelled.mat',
        # Backup: We'll use HuggingFace instead
    ]

    filepath = os.path.join(data_dir, 'nyu_depth_v2_labelled.mat')

    # Check if file exists and is valid
    if os.path.exists(filepath):
        file_size = os.path.getsize(filepath)
        if file_size > 100_000_000:  # Should be ~2.8GB
            print(f"Dataset file already exists ({file_size / 1e9:.1f}GB)")
            return data_dir
        else:
            print(f"Existing file is corrupted ({file_size} bytes). Removing...")
            os.remove(filepath)

    print("Original download is unreliable. Using HuggingFace instead...")
    return None  # Will trigger HuggingFace download

# NYU Depth V2 Dataset Class
class NYUDepthV2Dataset(Dataset):
    def __init__(self, data_dir, split='train', transform=None, target_transform=None, train_ratio=0.8):
        """
        NYU Depth V2 Dataset
        Args:
            data_dir: Directory containing nyu_depth_v2_labelled.mat
            split: 'train' or 'val'
            transform: Transform for RGB images
            target_transform: Transform for depth images
            train_ratio: Ratio for train/val split
        """
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform

        # Load the .mat file
        mat_file = os.path.join(data_dir, 'nyu_depth_v2_labelled.mat')
        if not os.path.exists(mat_file):
            print("Dataset not found. Downloading...")
            download_nyu_depth_v2(data_dir)

        print("Loading NYU Depth V2 dataset...")
        with h5py.File(mat_file, 'r') as f:
            # Load images and depths
            self.images = np.array(f['images'])  # Shape: (3, 480, 640, 1449)
            self.depths = np.array(f['depths'])  # Shape: (480, 640, 1449)

        # Transpose to (N, H, W, C) format
        self.images = np.transpose(self.images, (3, 1, 2, 0))  # (1449, 480, 640, 3)
        self.depths = np.transpose(self.depths, (2, 0, 1))     # (1449, 480, 640)

        # Create train/val split
        indices = list(range(len(self.images)))
        train_indices, val_indices = train_test_split(
            indices, train_size=train_ratio, random_state=42
        )

        if split == 'train':
            self.indices = train_indices
        else:
            self.indices = val_indices

        print(f"Loaded {len(self.indices)} {split} samples")

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]

        # Get RGB image and depth
        rgb = self.images[real_idx]  # (480, 640, 3)
        depth = self.depths[real_idx]  # (480, 640)

        # Convert to PIL Images
        rgb_image = Image.fromarray(rgb.astype(np.uint8))

        # Normalize depth to 0-255 range for PIL compatibility
        depth_normalized = ((depth - depth.min()) / (depth.max() - depth.min()) * 255).astype(np.uint8)
        depth_image = Image.fromarray(depth_normalized, mode='L')

        # Apply transforms
        if self.transform:
            rgb_image = self.transform(rgb_image)
        if self.target_transform:
            depth_image = self.target_transform(depth_image)

        return rgb_image, depth_image

# U-Net CNN Architecture for Depth Estimation
class DepthEstimationCNN(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(DepthEstimationCNN, self).__init__()

        # Encoder (Downsampling path)
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder (Upsampling path)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(128, 64)

        # Final layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

        # Pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))

        # Decoder with skip connections
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.dec1(dec1)

        # Final prediction
        output = self.final_conv(dec1)
        return torch.sigmoid(output)

# Loss function combining L1 and gradient loss
class DepthLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(DepthLoss, self).__init__()
        self.alpha = alpha

    def forward(self, pred, target):
        # L1 loss
        l1_loss = F.l1_loss(pred, target)

        # Gradient loss for edge preservation
        grad_loss = self.gradient_loss(pred, target)

        return l1_loss + self.alpha * grad_loss

    def gradient_loss(self, pred, target):
        # Compute gradients
        pred_grad_x = torch.abs(pred[:, :, :, :-1] - pred[:, :, :, 1:])
        pred_grad_y = torch.abs(pred[:, :, :-1, :] - pred[:, :, 1:, :])

        target_grad_x = torch.abs(target[:, :, :, :-1] - target[:, :, :, 1:])
        target_grad_y = torch.abs(target[:, :, :-1, :] - target[:, :, 1:, :])

        grad_loss_x = F.l1_loss(pred_grad_x, target_grad_x)
        grad_loss_y = F.l1_loss(pred_grad_y, target_grad_y)

        return grad_loss_x + grad_loss_y

# Training function
def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=0.001, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.to(device)
    print(f"Training on {device}")

    criterion = DepthLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0

        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for rgb, depth in train_pbar:
            rgb, depth = rgb.to(device), depth.to(device)

            optimizer.zero_grad()
            outputs = model(rgb)
            loss = criterion(outputs, depth)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')
            for rgb, depth in val_pbar:
                rgb, depth = rgb.to(device), depth.to(device)
                outputs = model(rgb)
                loss = criterion(outputs, depth)
                val_loss += loss.item()
                val_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

        # Learning rate scheduling
        scheduler.step(avg_val_loss)

        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_nyu_depth_model.pth')
            print(f'Best model saved with validation loss: {avg_val_loss:.4f}')

    return train_losses, val_losses

# Visualization function
def visualize_predictions(model, dataset, device, num_samples=4):
    """Visualize model predictions"""
    model.eval()
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 4*num_samples))

    with torch.no_grad():
        for i in range(num_samples):
            rgb, depth_gt = dataset[i]
            rgb_input = rgb.unsqueeze(0).to(device)
            depth_pred = model(rgb_input).squeeze().cpu()

            # Convert tensors to numpy for visualization
            rgb_np = rgb.permute(1, 2, 0).numpy()
            rgb_np = (rgb_np - rgb_np.min()) / (rgb_np.max() - rgb_np.min())
            depth_gt_np = depth_gt.squeeze().numpy()
            depth_pred_np = depth_pred.numpy()

            # Plot RGB, Ground Truth Depth, Predicted Depth
            axes[i, 0].imshow(rgb_np)
            axes[i, 0].set_title('RGB Image')
            axes[i, 0].axis('off')

            axes[i, 1].imshow(depth_gt_np, cmap='viridis')
            axes[i, 1].set_title('Ground Truth Depth')
            axes[i, 1].axis('off')

            axes[i, 2].imshow(depth_pred_np, cmap='viridis')
            axes[i, 2].set_title('Predicted Depth')
            axes[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

# Main training script
def main():
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    # Download and prepare dataset
    data_dir = './nyu_depth_v2'
    download_nyu_depth_v2(data_dir)

    # Data transforms
    transform = transforms.Compose([
        transforms.Resize((240, 320)),  # Resize for faster training
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    target_transform = transforms.Compose([
        transforms.Resize((240, 320)),
        transforms.ToTensor()
    ])

    # Create datasets
    train_dataset = NYUDepthV2Dataset(
        data_dir=data_dir,
        split='train',
        transform=transform,
        target_transform=target_transform
    )

    val_dataset = NYUDepthV2Dataset(
        data_dir=data_dir,
        split='val',
        transform=transform,
        target_transform=target_transform
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")

    # Initialize model
    model = DepthEstimationCNN()

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")

    # Train the model
    print("Starting training...")
    train_losses, val_losses = train_model(
        model,
        train_loader,
        val_loader,
        num_epochs=50,
        learning_rate=0.001
    )

    # Plot training curves
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    # Load best model and visualize results
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.load_state_dict(torch.load('best_nyu_depth_model.pth', map_location=device))

    print("Visualizing predictions...")
    visualize_predictions(model, val_dataset, device)

    plt.show()

# Function for inference on new images
def predict_depth(model_path, image_path, device=None):
    """Predict depth for a single image"""
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load model
    model = DepthEstimationCNN()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Load and preprocess image
    transform = transforms.Compose([
        transforms.Resize((240, 320)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Predict depth
    with torch.no_grad():
        depth_pred = model(image_tensor)
        depth_pred = depth_pred.squeeze().cpu().numpy()

    return depth_pred

if __name__ == "__main__":
    main()

Original download is unreliable. Using HuggingFace instead...
Dataset not found. Downloading...
Original download is unreliable. Using HuggingFace instead...
Loading NYU Depth V2 dataset...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './nyu_depth_v2/nyu_depth_v2_labelled.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)